In [1]:
import pandas as pd
from Acesso import Login
from Query import Query

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

pd.set_option('float_format','{:2f}'.format)

In [2]:
querys={
    
    'Vendas':
    
    """
    
    SELECT * FROM netfeira.vw_consolidado_venda


    """,
    
    'Compra':
    
    """

    SELECT * FROM netfeira.vw_sellin
    WHERE Tipo IN('PADRAO','BONIFICADA')    
    
    """
}

In [3]:
def Classificacao(val):
    
    if val<=80:
        
        tipo='A'
        
        pass
    
    elif val<=95:
        
        tipo='B'
        
        pass
    
    else:
        
        tipo='C'
        
        pass
    
    return tipo
    
    pass

In [ ]:
df=sql.CriarTabela(kwargs=querys)

In [ ]:
df['Vendas'].columns

In [ ]:
df['Vendas']=df['Vendas'].loc[~df['Vendas']['Fabricante'].isin(['DPA','NESTLE'])]

In [ ]:
tabela='Anual Empresa'

df['Temp']=df['Vendas']

df[tabela]=df['Temp'].groupby(['Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

df[tabela]['Pedidos']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'Pedido'].unique().tolist()))

df[tabela]['Clientes']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'ID Cliente'].unique().tolist()))

df[tabela]['MIX']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'SKU'].unique().tolist()))

df[tabela]['Abertura']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[(df['Temp']['Ano']==info)&(df['Temp']['Abertura']=='S'),'ID Cliente'].unique().tolist()))

df[tabela]['Ticket Médio']=round(df[tabela]['Total Venda']/df[tabela]['Pedidos'],2)

df[tabela]['Real KG']=round(df[tabela]['Total Venda']/df[tabela]['Peso Bruto KG'],2)

df[tabela]['Média Pedidos']=round(df[tabela]['Pedidos']/df[tabela]['Clientes'],2)

anos=df[tabela]['Ano'].unique().tolist()

colunas=df[tabela].columns[1:]

#ano x ano
for a in anos:

    for c in colunas:
        
        try:
    
            total_ant=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

            total_atu=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

            perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0
            
            df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc
            
            pass
        
        except:
            
            continue
        
        pass
    
    pass

#ano atual x ano
for a in anos:

    for c in colunas:
        
        try:
    
            total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

            total_atu=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

            perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0
            
            df[tabela].loc[df[tabela]['Ano']==a,f'Perc Anual {c}']=perc
            
            pass
        
        except:
            
            continue
        
        pass
    
    pass

df[tabela]

In [ ]:
tabela='Anual Canais'

grupos=df['Vendas']['Canal'].unique().tolist()

temp_df=pd.DataFrame()

for g in grupos:
    
    print(f'Fabricante: {g}')
    
    df['Temp']=df['Vendas'].loc[df['Vendas']['Canal']==g]

    df[tabela]=df['Temp'].groupby(['Canal','Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

    df[tabela]['Pedidos']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'Pedido'].unique().tolist()))

    df[tabela]['Clientes']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'ID Cliente'].unique().tolist()))

    df[tabela]['MIX']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'SKU'].unique().tolist()))

    df[tabela]['Abertura']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[(df['Temp']['Ano']==info)&(df['Temp']['Abertura']=='S'),'ID Cliente'].unique().tolist()))

    df[tabela]['Ticket Médio']=round(df[tabela]['Total Venda']/df[tabela]['Pedidos'],2)

    df[tabela]['Real KG']=round(df[tabela]['Total Venda']/df[tabela]['Peso Bruto KG'],2)

    df[tabela]['Média Pedidos']=round(df[tabela]['Pedidos']/df[tabela]['Clientes'],2)

    anos=df[tabela]['Ano'].unique().tolist()

    colunas=df[tabela].columns[1:]

    #ano x ano
    for a in anos:

        for c in colunas:

            try:

                total_ant=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                total_atu=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                pass

            except:

                continue

            pass

        pass

    #ano atual x ano
    for a in anos:

        for c in colunas:

            try:

                total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                total_atu=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

                perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc Anual {c}']=perc

                pass

            except:

                continue

            pass

        pass
    
    temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)

    display(df[tabela]) 
    
    pass

df[tabela]=temp_df

In [ ]:
ano_atual=df['Vendas']['Ano'].max()

In [ ]:
tabela='Rep Canal'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

df[tabela]=df['Temp'].groupby(['Canal'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

colunas=df[tabela].columns[1:]

for c in colunas:
    
    total=df[tabela][c].sum()
    
    for i in range(0,len(df[tabela])):
        
        df[tabela].loc[i,f'Perc {c}']=round(df[tabela].loc[i,c].sum()/total,4)*100
        
        pass
    
    pass

df[tabela]

In [ ]:
tabela='Anual Grupo DM'

grupos=df['Vendas']['Grupo Indústria'].loc[df['Vendas']['Grupo Indústria']!='OUTRAS MARCAS'].unique().tolist()

temp_df=pd.DataFrame()

for g in grupos:
    
    print(f'Fabricante: {g}')
    
    df['Temp']=df['Vendas'].loc[df['Vendas']['Fabricante']==g]

    df[tabela]=df['Temp'].groupby(['Grupo Indústria','Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

    df[tabela]['Pedidos']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'Pedido'].unique().tolist()))

    df[tabela]['Clientes']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'ID Cliente'].unique().tolist()))

    df[tabela]['MIX']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'SKU'].unique().tolist()))

    df[tabela]['Abertura']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[(df['Temp']['Ano']==info)&(df['Temp']['Abertura']=='S'),'ID Cliente'].unique().tolist()))

    df[tabela]['Ticket Médio']=round(df[tabela]['Total Venda']/df[tabela]['Pedidos'],2)

    df[tabela]['Real KG']=round(df[tabela]['Total Venda']/df[tabela]['Peso Bruto KG'],2)

    df[tabela]['Média Pedidos']=round(df[tabela]['Pedidos']/df[tabela]['Clientes'],2)

    anos=df[tabela]['Ano'].unique().tolist()

    colunas=df[tabela].columns[1:]

    #ano x ano
    for a in anos:

        for c in colunas:

            try:

                total_ant=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                total_atu=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                pass

            except:

                continue

            pass

        pass

    #ano atual x ano
    for a in anos:

        for c in colunas:

            try:

                total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                total_atu=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

                perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                df[tabela].loc[df[tabela]['Ano']==a,f'Perc Anual {c}']=perc

                pass

            except:

                continue

            pass

        pass
    
    temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)

    display(df[tabela]) 
    
    pass

df[tabela]=temp_df

In [ ]:
tabela='Canal x Marca'

grupos=df['Vendas']['Grupo Indústria'].loc[df['Vendas']['Grupo Indústria']!='OUTRAS MARCAS'].unique().tolist()

canais=df['Vendas']['Canal'].unique().tolist()

temp_df=pd.DataFrame()

for g in grupos:
    
    for c in canais:
    
        print(f'Fabricante: {g} x Canal: {c}')

        df['Temp']=df['Vendas'].loc[(df['Vendas']['Fabricante']==g)&(df['Vendas']['Canal']==c)]

        df[tabela]=df['Temp'].groupby(['Grupo Indústria','Canal','Ano'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

        df[tabela]['Pedidos']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'Pedido'].unique().tolist()))

        df[tabela]['Clientes']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'ID Cliente'].unique().tolist()))

        df[tabela]['MIX']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[df['Temp']['Ano']==info,'SKU'].unique().tolist()))

        df[tabela]['Abertura']=df[tabela]['Ano'].apply(lambda info: len(df['Temp'].loc[(df['Temp']['Ano']==info)&(df['Temp']['Abertura']=='S'),'ID Cliente'].unique().tolist()))

        df[tabela]['Ticket Médio']=round(df[tabela]['Total Venda']/df[tabela]['Pedidos'],2)

        df[tabela]['Real KG']=round(df[tabela]['Total Venda']/df[tabela]['Peso Bruto KG'],2)

        df[tabela]['Média Pedidos']=round(df[tabela]['Pedidos']/df[tabela]['Clientes'],2)

        anos=df[tabela]['Ano'].unique().tolist()

        colunas=df[tabela].columns[1:]

        #ano x ano
        for a in anos:

            for c in colunas:

                try:

                    total_ant=df[tabela].loc[df[tabela]['Ano']==a-1,c].sum()

                    total_atu=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                    perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                    df[tabela].loc[df[tabela]['Ano']==a,f'Perc {c}']=perc

                    pass

                except:

                    continue

                pass

            pass

        #ano atual x ano
        for a in anos:

            for c in colunas:

                try:

                    total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()

                    total_atu=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()

                    perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0

                    df[tabela].loc[df[tabela]['Ano']==a,f'Perc Anual {c}']=perc

                    pass

                except:

                    continue

                pass

            pass

        temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)

        display(df[tabela])
        
        pass
    
    pass

df[tabela]=temp_df

In [ ]:
tabela='Rep Fabricante'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

df[tabela]=df['Temp'].groupby(['Fabricante'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

colunas=df[tabela].columns[1:]

for c in colunas:
    
    total=df[tabela][c].sum()
    
    for i in range(0,len(df[tabela])):
        
        df[tabela].loc[i,f'Perc {c}']=round(df[tabela].loc[i,c].sum()/total,4)*100
        
        pass
    
    pass

df[tabela].sort_values('Total Venda',ascending=False,ignore_index=True,inplace=True)

df[tabela]

In [ ]:
tabela='Rep Canal x Fabricante'

temp_df=pd.DataFrame()

grupos=df['Rep Fabricante']['Fabricante'].head().unique().tolist()

canais=df['Rep Canal']['Canal'].unique().tolist()

for g in grupos:
    
    for c in canais:

        df['Temp']=df['Vendas'].loc[(df['Vendas']['Ano']==ano_atual)&(df['Vendas']['Fabricante']==g)&(df['Vendas']['Canal']==c)]

        df[tabela]=df['Temp'].groupby(['Fabricante','Canal'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

        colunas=df[tabela].columns[1:]
        
        temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
                
        pass
    
    pass

df[tabela]=temp_df

colunas=df[tabela].columns[2:]

for g in grupos:
    
    for c in colunas:
    
        df['Temp']=df[tabela].loc[df[tabela]['Fabricante']==g]
        
        for i in df['Temp'].index.tolist():
            
            total=df['Temp'][c].sum()
            
            df[tabela].loc[i,f'Perc {c}']=round(df[tabela].loc[i,c].sum()/total,4)*100
            
            pass
        
        pass
    
    pass

df[tabela]

In [ ]:
tabela='Rep Linha x Canal'

temp_df=pd.DataFrame()

grupos=df['Rep Fabricante']['Fabricante'].head().unique().tolist()

canais=df['Rep Canal']['Canal'].unique().tolist()

for g in grupos:
    
    df['Vendas'].loc[df['Vendas']['Fabricante']==g].unique().tolist()
    
    for c in canais:

        df['Temp']=df['Vendas'].loc[(df['Vendas']['Ano']==ano_atual)&(df['Vendas']['Fabricante']==g)&(df['Vendas']['Canal']==c)]

        df[tabela]=df['Temp'].groupby(['Fabricante','Canal'],as_index=False).agg({'Total Venda':'sum','Peso Bruto KG':'sum'})

        colunas=df[tabela].columns[1:]
        
        temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
                
        pass
    
    pass

df[tabela]=temp_df

colunas=df[tabela].columns[2:]

for g in grupos:
    
    for c in colunas:
    
        df['Temp']=df[tabela].loc[df[tabela]['Fabricante']==g]
        
        for i in df['Temp'].index.tolist():
            
            total=df['Temp'][c].sum()
            
            df[tabela].loc[i,f'Perc {c}']=round(df[tabela].loc[i,c].sum()/total,4)*100
            
            pass
        
        pass
    
    pass

df[tabela]

In [ ]:
colunas=['Fabricante']

tabela='Fabricante ABC'

col='Total Venda'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

In [ ]:
colunas=['Fabricante','Linha']

tabela='Linha ABC'

col='Total Venda'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

In [ ]:
colunas=['Fabricante','Linha','Produto']

tabela='Produtos ABC'

col='Total Venda'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

In [ ]:
colunas=['SKU']

tabela='Lista ABC'

col='Total Venda'

df['Temp']=df['Vendas'].loc[df['Vendas']['Ano']==ano_atual]

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

In [ ]:
col='Peso Bruto KG'

codigos=df['Lista ABC']['SKU'].unique().tolist()

tabela='Queda'

df['Temp']=df['Vendas'].loc[df['Vendas']['SKU'].isin(codigos)]

df[tabela]=df['Temp'].groupby(['SKU','Produto','Fabricante','Linha','Ano'],as_index=False).agg({col:'sum'})

df[tabela]=df[tabela].pivot(index=['SKU','Produto','Fabricante','Linha'],columns='Ano',values=col).reset_index()

anos=df[tabela].columns[1:]

for a in anos:
    
    df[tabela].loc[df[tabela][a].isnull(),a]=0
    
    pass

for i in range(0,len(df[tabela])):

    for a in anos:
        
        try:
        
            total_ant=df[tabela].loc[i,a].sum()
            
            total_atu=df[tabela].loc[i,a+1].sum()
            
            perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0
            
            df[tabela].loc[i,f'Perc {a+1}']=perc
            
            pass
        
        except:
            
            continue

        pass
    
    pass

colunas=[l for l in df[tabela] if str(l).find('Perc')>=0]


for i in range(0,len(df[tabela])):

    lista=df[tabela].loc[i,colunas].tolist()
    
    lista.reverse()
    
    count=[l for l in lista[:2] if l>0]
    
    if len(count)<=0:
        
        
        tipo='QUEDA'
        
        pass
    
    elif len(count)==1:
        
        tipo='ACOMPANHAR'
        
        pass
    
    else:
        
        tipo='CRESCIMENTO'
        
        pass
    
    
    df[tabela].loc[i,'Desempenho']=tipo
    
    pass

df[tabela]=df[tabela].merge(df['Lista ABC'][['SKU','Classificação']],on='SKU',how='inner')

df[tabela]

### SELL IN

In [ ]:
df['Compra']['Ano']=df['Compra']['Data de Recebimento'].dt.year

In [ ]:
df['Compra'].columns

In [ ]:
colunas=['Fornecedor']

tabela='Fornecedor ABC'

col='Total NFe'

df['Temp']=df['Compra'].loc[df['Compra']['Ano']==ano_atual]

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

In [ ]:
tabela='Ano Compra'

df[tabela]=df['Compra'].loc[df['Compra']['Ano']<=ano_atual].groupby(['Ano'],as_index=False).agg({'Total NFe':'sum','Peso Bruto':'sum'})

colunas=df[tabela].columns[1:]

anos=df[tabela]['Ano'].unique().tolist()

for a in anos:
    
    for c in colunas:
        
        try:
        
            total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()
            
            total_atu=df[tabela].loc[df[tabela]['Ano']==a+1,c].sum()
            
            perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0
            
            df[tabela].loc[df[tabela]['Ano']==a+1,f'Perc {c}']=perc
                        
            pass
        
        except:
            
            continue
        
        pass
    
    pass

for a in anos:
    
    for c in colunas:
        
        try:
        
            total_ant=df[tabela].loc[df[tabela]['Ano']==a,c].sum()
            
            total_atu=df[tabela].loc[df[tabela]['Ano']==anos[-1],c].sum()
            
            perc=round((total_atu/total_ant)-1,4)*100 if total_ant>0 else 0
            
            df[tabela].loc[df[tabela]['Ano']==a,f'Anual {c}']=perc
                        
            pass
        
        except:
            
            continue
        
        pass
    
    pass

df[tabela]

In [ ]:
writer=pd.ExcelWriter('Consolidado.xlsx',engine='xlsxwriter')

for tabela,dados in df.items():
    
    if tabela in querys.keys() or tabela=='Temp':
                
        continue
    
    df[tabela].to_excel(writer,sheet_name=tabela,index=False)
    
    pass

writer.close()